In [ ]:
#!pip install igraph numpy scipy

In [ ]:
import igraph as ig
import numpy as np
from scipy.spatial import Delaunay
import math
import matplotlib.pyplot as plt
import plotly.express as px
from functools import reduce
import sys
import queue
import copy
from lib import SplitList, createTaskQueue, createMPPool, processaftereachother, printdebug, crdbg, UnionFind, AreaCache, TimeMeasure, parallelFun, ifprint, samplepairs
from distances import d_geodesic, d_euclidean, distanceFnFactory, d_mhOneDim
from graphfns import *
from libfgraphs import *
import itertools
import uuid
import pandas as pd

#gc.set_debug(gc.DEBUG_STATS)
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(x):
        return x


RNG = np.random.default_rng(seed=102671797463975) 


GLOBAL_SCALE_FACTOR = 1
GLOBAL_DISTANCE_FN = d_geodesic
def distance_GraphObj(g,v1,v2, distfn=GLOBAL_DISTANCE_FN):
    return distfn(g, v1, v2)

GLOBAL_DIMENSION=2
PRECISION=100

In [ ]:
def ExtensionalWeaklyConvexHull2(g, vertexset, theta, epsilon=0.0, distfn=distance_GraphObj, debug=False, areacache=None):
    markedVerticies = set()

    C = set()
    E = set()
    Q = queue.Queue()

    ac = areacache
    if areacache == None:
        ac = AreaCache(g, defaultdistfn=distfn)

    vertexset = vertextoindex(vertexset)
    for vertex in vertexset:
        markedVerticies.add(vertex)
        Q.put(vertex)
    
    while not Q.empty():
        el = Q.get()
        C.add(el)
        thetanh = ac.computeNHAreas(el, theta)

        for nhel in C.intersection(thetanh):
            E.add((el, nhel))

            zset = ac.computeZsets(el, nhel, epsilon=epsilon)
            zset = zset.difference(markedVerticies)

            for z in zset:
                markedVerticies.add(z)
                Q.put(z)
                    
    g.vs['mark'] = 0
    markedVerticies = indextovertex(g, markedVerticies)
    for v in markedVerticies:
        v['mark']=1
    return C,E

In [ ]:
from generatelineandgrid import generateTestLine, generateTestGrid
from OldExtensionalWeaklyConvexHull import ExtensionalWeaklyConvexHull


def AlgTest():
    def testline(numVerticies, numvertcl0=2, distance=3):
        if numvertcl0*2 > numVerticies:
            raise ValueError('numvertcl0*2 > numVerticies')
        testgraph = generateTestLine(numVerticies, numvertcl0)
        showgraph(testgraph, recursive=False, classmark=0)
        C,E = ExtensionalWeaklyConvexHull(testgraph, testgraph.vs.select(class_eq=0), distance, epsilon=0.0, distfn=d_step, debug=True)
        print(len(C), len(E))
        print([v.index for v in C])
        print([(e[0].index, e[1].index) for e in E])
        showgraph(testgraph, recursive=False, classmark=0)

    def testgrid(numwidth, numlen, sizecorners, distance=3):
        testgraph = generateTestGrid(numwidth, numlen, sizecorners)
       
        C,E = ExtensionalWeaklyConvexHull(testgraph, testgraph.vs.select(class_eq=0), distance, epsilon=0.0, distfn=d_step, debug=True)
        print(len(C), len(E))
        print([v.index for v in C])
        print([(e[0].index, e[1].index) for e in E])
        showgraph(testgraph, recursive=False, classmark=0)

        
    testline(7, numvertcl0=3, distance=3)
    testline(8, numvertcl0=3, distance=3)
    testgrid(8,7,3,distance=3)

#AlgTest()

In [ ]:

def plotcomponents(g, C, classmark=0, saveAs=None, noshow=False):
    g.vs['cluster'] = -1
    ret = getComponents(g, set(vertextoindex(C)))
    for i, cluster in enumerate(ret):
        cl = indextovertex(g, cluster)
        for v in cl:
            v['cluster'] = i

    ShowNumAttr(g, attr='cluster', continues=False, saveAs=saveAs, noshow=noshow)



def plotThetaComponents(g, distfn=d_step, epsilon=0.0, start=0, end=10, steps=100, plotintermediate=False, areaCache=None, showcluster=True, nocluster=-1, nopoints=-1):
    def getThetaComponents(g, theta, oldlen=-1):
        C,E = ExtensionalWeaklyConvexHull2(g, g.vs.select(class_eq=0), theta, epsilon=epsilon, distfn=distfn, debug=False, areacache=areaCache)
        C = set(vertextoindex(C))
        ret = getComponents(g, C)
        if plotintermediate and len(ret) != oldlen:
            name = "images/components_nopoints_{nopoints}_cluster_{cluster}_theta_{theta}_lenret_{rr}_{uuid}".format(nopoints=nopoints, cluster=nocluster, theta=round(theta,2), rr=len(ret), uuid=uuid.uuid4())
            showgraph(g, recursive=False, classmark=0, saveAs=name)
        return ret
    
    thetas = np.linspace(start, end, steps)
    print(thetas)
    sizes = []
    oldlen = -1
    ret = None
    for theta in tqdm(thetas):
        print("theta", theta)
        newret = getThetaComponents(g, theta, oldlen=oldlen)

        if showcluster and oldlen!=len(newret):
            g.vs['cluster'] = -1
            for i, cluster in enumerate(newret):
                cl = indextovertex(g, cluster)
                for v in cl:
                    v['cluster'] = i
            name = "images/components_nopoints_{nopoints}_cluster_{cluster}_theta_{theta}_lenret_{rr}_Cluster_{uuid}".format(nopoints=nopoints, cluster=nocluster, theta=round(theta,2), rr=len(newret), uuid=uuid.uuid4())
            ShowNumAttr(g, attr='cluster', continues=False, saveAs=name)



        if oldlen<len(newret) and ret != None:
            print("WARNING: oldlen<len(newret)")
            print(ret)
            print(newret)

        ret = newret
        oldlen = len(ret)
        sizes.append(oldlen)
    print(sizes)
    if plotintermediate:
        showgraph(g, recursive=False, classmark=0)

    fig = px.line(x=thetas, y=sizes)
    fig.update_layout(
        xaxis_title="Theta",
        yaxis_title="Number of Components",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="#7f7f7f"
        )
    )
    fig.write_image("images/_Components_nopoints_{nopoints}_cluster_{cluster}_{uuid}_.png".format(nopoints=nopoints, cluster=nocluster, uuid=uuid.uuid4()))
    fig.show()

        
        

def plotThetaSize(g, distfn=d_step, epsilon=0.0, start=0, end=10, steps=100, plotintermediate=False, areaCache=None, nocluster=-1, nopoints=-1):
    def getThetaSize(g, theta, oldlen=-1):
        C,E = ExtensionalWeaklyConvexHull2(g, g.vs.select(class_eq=0), theta, epsilon=epsilon, distfn=distfn, debug=False, areacache=areaCache)
        ret = len(C)
        if plotintermediate and ret != oldlen:
            name = "images/sizes_nopoints_{nopoints}_cluster_{cluster}_theta_{theta}_lenret_{rr}_{uuid}".format(nopoints=nopoints, cluster=nocluster, theta=round(theta,2), rr=ret, uuid=uuid.uuid4())
            showgraph(g, recursive=False, classmark=0, saveAs=name)
        return ret
    thetas = np.linspace(start, end, steps)
    print(thetas)
    sizes = []
    oldlen = -1
    for theta in tqdm(thetas):
        print("theta", theta)
        oldlen = getThetaSize(g, theta, oldlen=oldlen)
        sizes.append(oldlen)
    print(sizes)
    if plotintermediate:
        
        showgraph(g, recursive=False, classmark=0)

    fig = px.line(x=thetas, y=sizes)
    fig.update_layout(
        xaxis_title="Theta",
        yaxis_title="Size",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="#7f7f7f"
        )
    )
    fig.write_image("images/_Sizes_nopoints_{nopoints}_cluster_{cluster}_{uuid}_.png".format(nopoints=nopoints, cluster=nocluster,uuid=uuid.uuid4()))
    fig.show()


def testGdis(n=300, ccluster=4):
    localdistfn = d_step
    global GLOBAL_SCALE_FACTOR
    global GLOBAL_DIMENSION
    global PRECISION

    testgraph = generateRandomGraph(n, coordinateGenerator=generateRandomCoordinatesShifted, deleteEdges=0)

    testgraph = generateGraphWithOrigins(testgraph, ccluster)
    showgraph(testgraph)

    ac = AreaCache(testgraph, defaultdistfn=localdistfn)
    ac.precomputeDistances()
    ac.precomputeZsets(epsilon=0.0)


    plotThetaComponents(testgraph, distfn=localdistfn, steps=10, start=0.1, end=10.1,  plotintermediate=True, areaCache=ac, nopoints=n, nocluster=ccluster)
    plotThetaSize(testgraph, distfn=localdistfn, steps=10, start=0.1, end=10.1,  plotintermediate=True, areaCache=ac, nopoints=n, nocluster=ccluster)

testGdis()

In [ ]:
import gc

from Measures import convSanity1, convSanity2sampled, gconfm1A, gconfm1B, gconfm1Aminmax, EntropyConvexity


class convMeasureTester:
    n = 300
    testac = None

    maxn = 1000
    scaledfactor = 1
    dimension = 2
    precision = 2
    localdistfn = d_step
    testgraph = None
    ac = None
    maxlen = 0
    minvalue = 0
    maxvalue = 0
    testspervalue = 0
    graph = None
    testgraphen = {}
    coloryourself = True
    

    def __init__(self, maxn=1000, precompute=False, usegraph=None, localdistfn=None) -> None:
        global GLOBAL_SCALE_FACTOR
        global GLOBAL_DIMENSION
        global PRECISION

    
        self.localdistfn = d_step if localdistfn is None else localdistfn
        self.scaledfactor = GLOBAL_SCALE_FACTOR
        self.dimension = GLOBAL_DIMENSION
        self.precision = PRECISION
        

        self.graph = generateRandomGraph(maxn, coordinateGenerator=generateRandomCoordinatesShifted, deleteEdges=0.05) if usegraph is None else usegraph
        self.coloryourself = True if usegraph is None else False

        self.maxn = self.graph.vcount()

        self.ac = AreaCache(self.graph, defaultdistfn=self.localdistfn)

        self.ac.precomputeLongestSP()
        self.maxlen = self.ac.getLongestPossiblePath()
        print("maxlen", self.maxlen)
        if precompute:
            self.ac.precomputeDistances()
            for l in range(1, self.maxlen+1):
                self.ac.precomputeNHAreas(theta=float(l))
        gc.collect()


    def setToKey(self, s):
        return str(sorted(s))
    
    def testinit(self, n=100, useexisting=False, precompute=False):
        if useexisting:
            for key in self.testgraphen:
                if self.testgraphen[key]['len'] == n:
                    self.n = n
                    self.testgraph = self.testgraphen[key]['graph']
                    self.testac = self.testgraphen[key]['ac']
                    return
        
        if self.ac is None:
            self.ac = AreaCache(self.graph, defaultdistfn=self.localdistfn)
            #self.ac.precomputeDistances()


        vs = vertextoindex(self.graph.vs)
        vs = RNG.choice(vs, size=n, replace=False).tolist()

        SKey = self.setToKey(vs)
        newtestgraph = {}
        newtestgraph['len'] = len(vs)

        newtestgraph = {}
        newtestgraph['vs'] = vs
        newtestgraph['graph'] = self.graph.subgraph(vs)
        
        newdc, o_n, n_o = self.ac.getDisCache().subset(vs)
        if precompute:
            newtestgraph['ac'] = AreaCache(newtestgraph['graph'], defaultdistfn=self.localdistfn, distancecache=newdc)
        else:
            newtestgraph['ac'] = AreaCache(newtestgraph['graph'], defaultdistfn=self.localdistfn)
        newtestgraph['on'] = o_n
        newtestgraph['no'] = n_o

        print(o_n)
        print(n_o)

        newtestgraph['ac'].precomputeLongestSP()
        print("precomputeSP done")

        if precompute:

            _nhareas = self.ac._getNHAreas()
            for el in _nhareas:
                if not el in o_n:
                    continue
                for theta in _nhareas[el]:
                    newtestgraph['ac'].setNHAreas(o_n[el], theta, set([o_n[elx] for elx in _nhareas[el][theta] if elx in o_n]))
            
            print("NHAreas done")

            _nhseareas = self.ac._getNHSEAreas()
            for el in _nhseareas:
                if not el in o_n:
                    continue
                for theta in _nhseareas[el]:
                    newtestgraph['ac'].setNHSEAreas(o_n[el], theta, set([o_n[elx] for elx in _nhseareas[el][theta] if elx in o_n]))

            print("NHSEAreas done")

            _deltaareas = self.ac._getDeltaAreas()
            for vx in _deltaareas:
                for vy in _deltaareas[vx]:
                    if not vx in o_n or not vy in o_n:
                        continue
                    newtestgraph['ac'].setDeltaAreas(o_n[vx], o_n[vy], set([o_n[el] for el in _deltaareas[vx][vy] if el in o_n]))
            
            print("DeltaAreas done")
            
            

            newtestgraph['ac'].precomputeZsets(epsilon=0.0)

            print("precomputeZsets done")


        self.n = n
        self.testgraph = newtestgraph['graph']
        self.testac = newtestgraph['ac']

        self.testgraphen[SKey] = newtestgraph

        #showgraph(self.testgraph, recursive=False, classmark=0)


    def copy(self):
        mycopy = copy.deepcopy(self)
        return mycopy

    def generatevorigins(self, ccluster):


        vorigins = []
        def createco():
            co = (RNG.random(size=(self.dimension))-0.5)*self.scaledfactor
            co = np.around(co, decimals=self.precision)
            co = { "coordinates": co.tolist(), "sigmaDist": 0.1*self.scaledfactor, "originprob": 1 }
            return co

        for _ in range(ccluster):
            co = createco()
            
            while any([ d_euclidean(self.testgraph, co, originI) < 0.15*self.scaledfactor for originI in vorigins]):
                co = createco()

            vorigins.append(co)
        return vorigins
    
    def stepgraph(self, givengraph, setA, convexehull, i=0, areacache=None, ldfn=None, workerno=0, chduration=0, numberOfOrigins=0):
            if ldfn is None:
                ldfn = d_step
            areac = areacache
            if areac is None:
                areac = AreaCache(givengraph, defaultdistfn=ldfn)
            returnobj = {}
            returnobj['Theta']=i

            sp = 1
            TM = TimeMeasure("TM")
            returnobj['convSanity1']=convSanity1(setA, convexehull)
            returnobj['convSanity1time'] = TM.renew()+chduration

            returnobj['convSanity2']=convSanity2sampled(givengraph, setA, convexehull, samplepercentage=sp)
            returnobj['convSanity2time'] = TM.renew()

            returnobj['gconfm1A']=gconfm1A(givengraph, setA, samplepercentage=sp)
            returnobj['gconfm1Atime'] = TM.renew()

            returnobj['gconfm1B']=gconfm1B(givengraph, setA, samplepercentage=sp)
            returnobj['gconfm1Btime'] = TM.renew()

            returnobj['gconfm1Aminmax']=gconfm1Aminmax(givengraph, setA, samplepercentage=sp)
            returnobj['gconfm1Aminmaxtime'] = TM.renew()

            returnobj['EntropyConvexityDist']=EntropyConvexity(givengraph, setA, delta=float(self.maxlen), distfn=ldfn, samplepercentage=sp)
            returnobj['EntropyConvexityDisttime'] = TM.renew()
          
            returnobj['NumberOfPoints']=len(setA)
            returnobj['NumberOfComponents']=len(getComponents(givengraph, setA))
            returnobj['n']=self.n

            returnobj['NumberOfOrigins']=numberOfOrigins

            return returnobj

    def createdataframesSubParallel(self, argumentsobj):
        leni, givengraph, setA, areac, convexehull, chduration,  ldfn, workerno, numberOfOrigins = argumentsobj
        print("Worker started, createdataframesSubParallel leni={leni}".format(leni=leni))
        CH, _ = ExtensionalWeaklyConvexHull2(givengraph, setA, float(leni), epsilon=0.00, distfn=ldfn, areacache=areac)
        lresults = self.stepgraph(givengraph, CH, convexehull, i=leni, areacache=areac, ldfn=ldfn, workerno=workerno, chduration=chduration, numberOfOrigins=numberOfOrigins)
        print("Worker ended, createdataframesSubParallel leni={leni}".format(leni=leni))
        return lresults
    
    def createdataframes(self, iteratorarg):
        givengraph, setA, areacache, ldfn, workerno, numberOfOrigins, subparallel = iteratorarg
        try:
            ifprint(not subparallel, "Worker {w} started".format(w=workerno))
            print("Arguments",workerno, len(setA), givengraph.vcount(), len(givengraph.vs.select(class_eq=0)), len(givengraph.vs.select(class_eq=1)))
            if ldfn is None:
                ldfn = d_step
            areac = areacache
            if areac is None:
                areac = AreaCache(givengraph, defaultdistfn=ldfn)


            chtm = TimeMeasure("CH")
            convexehull, _ = ExtensionalWeaklyConvexHull2(givengraph, setA, float(self.maxlen), epsilon=0.00, distfn=ldfn)
            chduration = chtm.equite()
            
            givengraph.vs['mark'] = 0


            results = []
            results.append(self.stepgraph(givengraph, setA, convexehull, i=0, areacache=areac, ldfn=ldfn, workerno=workerno, chduration=chduration, numberOfOrigins=numberOfOrigins))
            
            lmlen = areac.getLongestPossiblePath(setA)


            def nequal(a,b):
                return math.isclose(a, b, rel_tol=1e-09, abs_tol=0.0)
            
            if subparallel:
                pfc = parallelFun()
                iterarg = [ (leni, givengraph, setA, areac, convexehull, chduration,  ldfn, workerno, numberOfOrigins) for leni in range(1, lmlen+1)]
                results = results + pfc.run(self.createdataframesSubParallel, iterarg)
            else:
                endsum = 0
                for i in tqdm(range(1, lmlen+1), desc="\n Worker {w} \t".format(w=workerno)):
                    lresults = None

                    if endsum >= 2:
                        lresults = results[-1]
                        lresults['Theta']=i
                    else:
                        CH, _ = ExtensionalWeaklyConvexHull2(givengraph, setA, float(i), epsilon=0.00, distfn=ldfn, areacache=areac)
                        lresults = self.stepgraph(givengraph, CH, convexehull, i=i, areacache=areac, ldfn=ldfn, workerno=workerno, chduration=chduration, numberOfOrigins=numberOfOrigins)
                    
                    results.append(lresults)
                    givengraph.vs['mark'] = 0   

            ifprint(not subparallel, "Worker {w} done".format(w=workerno))
            return results, areacache
        except Exception as e:
            ifprint(not subparallel, "Worker {w} failed".format(w=workerno))
            print(e)
            return None, areacache
    
    def iteratorargFn(self, i, copydata=False, subparallel=False):
        n = self.n
        numberOfOrigins = int(math.floor(i/self.testspervalue))+self.minvalue
        trialgraph = copy.deepcopy(self.testgraph) if copydata else self.testgraph
        if self.coloryourself:
            trialgraph = genRandPartitioningOriginProb(trialgraph, vorigins=self.generatevorigins(numberOfOrigins))
        areac = copy.deepcopy(self.testac) if copydata else self.testac
        ldfn = copy.deepcopy(self.localdistfn) if copydata else self.localdistfn
        print("Iterator {i} started: trialgraph ({n0}+{n1}={nC}) ({eCount}); noorigins={_i}".format(i=i, n0=len(trialgraph.vs.select(class_eq=0)), n1=len(trialgraph.vs.select(class_eq=1)), nC=trialgraph.vcount(), _i=numberOfOrigins, eCount=trialgraph.ecount()))
        return trialgraph, vertextoindex(trialgraph.vs.select(class_eq=0)), areac, ldfn, i+1, numberOfOrigins, subparallel
    
    def test(self, minvalue=3, maxvalue=6, testspervalue=3, n=100, poolfn=processaftereachother, showresults=False, runparallel=True): #maxval7
        gc.collect()
        print(gc.garbage)
        self.minvalue = minvalue
        self.maxvalue = maxvalue
        self.testspervalue = testspervalue
        numberoftasks = int((maxvalue-minvalue)*testspervalue)

        self.testinit(n=n)

        wo = self.copy()

        print("Starting Pool, total tasks: {w} \n\n".format(w=numberoftasks))
        pfc = parallelFun()
        subparallel=False if runparallel else True
        itearg = [wo.iteratorargFn(i, subparallel=subparallel) for i in range(numberoftasks)]
        print(itearg)
        if runparallel:
            res = pfc.run(wo.createdataframes, itearg)
        else:
            res = []
            TM = TimeMeasure("TM")
            for i in tqdm(itearg):
                restp = wo.createdataframes(i)
                res.append(restp)



        print("Finished Pool \n\n")

        wo.testgraph = None
        wo.testac = None
        wo.graph = None
        wo.ac = None
        wo.localdistfn = None
        wo.testgraphen = {}
        del wo ##?
        gc.collect()


        times = [r for _,r in res]

        print(times)


        res = [r for r in res if r is not None]

        ret = [pd.DataFrame(r) for r,_ in res if r is not None]

        
        if ret is None or len(ret) == 0:
            ret = pd.DataFrame()
        
        if len(ret) == 1:
            ret = ret[0]

        if not ret is None and not len(ret)<=1 and not isinstance(ret, pd.DataFrame):
            ret = pd.concat(ret, ignore_index=True)


        return ret
    

In [ ]:
poolfn = createMPPool

def testgraphN(graph, cummulateddf, ccluster=None):
    if not ccluster is None:
        graph = generateGraphWithOrigins(graph, ccluster=ccluster, dimension=GLOBAL_DIMENSION, precision=PRECISION)
    else:
        ccluster = 2
    cmtst = convMeasureTester(usegraph=graph)
    novert = graph.vcount()
    resDf = cmtst.test(n=novert, minvalue=ccluster, maxvalue=ccluster+1, testspervalue=1, runparallel=False)
    cummulateddf = pd.concat([cummulateddf, resDf])
    return resDf, cummulateddf



cumuDf = pd.DataFrame()

import os

In [ ]:
""" TEST for Grid """

myuuid = uuid.uuid4()

if not os.path.exists('./shr'):
    os.makedirs('./shr')

if not os.path.exists('./shr/grid'):
    os.makedirs('./shr/grid')

g2 = generateTestGrid(6,5,2)
g2ac = AreaCache(g2, defaultdistfn=d_step, pcdistances=True)

for l in range(1, 10):
    C,E = ExtensionalWeaklyConvexHull2(g2, g2.vs.select(class_eq=0), l, epsilon=0.0, distfn=d_step, debug=False, areacache=g2ac)
    print(len(C), len(E))
    showgraph(g2, classmark=0, recursive=False, saveAs="shr/grid/grid_{l}_{uuid}.png".format(l=l, uuid=myuuid), noshow=True)
    plotcomponents(g2, C, classmark=0, saveAs="shr/grid/grid_comp_{l}_{uuid}.png".format(l=l, uuid=myuuid), noshow=True)

result_,cumuDf =  testgraphN(g2, cumuDf)
result_.to_pickle("shr/grid/grid_df_{uuid}.pkl".format(uuid=myuuid))
save_graph(g2, "shr/grid/grid_graph_{uuid}.pkl".format(uuid=myuuid))


In [ ]:
""" TEST for Line """

myuuid = uuid.uuid4()

if not os.path.exists('./shr'):
    os.makedirs('./shr')

if not os.path.exists('./shr/line'):
    os.makedirs('./shr/line')

g3 = generateTestLine(20, 5)
g3ac = AreaCache(g3, defaultdistfn=d_step, pcdistances=True)
for l in range(1, 15):
    C,E = ExtensionalWeaklyConvexHull2(g3, g3.vs.select(class_eq=0), l, epsilon=0.0, distfn=d_step, debug=False, areacache=g3ac)
    print(len(C), len(E))
    showgraph(g3, classmark=0, recursive=False, saveAs="shr/line/line_{l}_{uuid}.png".format(l=l, uuid=myuuid), noshow=True)
    plotcomponents(g3, C, classmark=0, saveAs="shr/line/line_comp_{l}_{uuid}.png".format(l=l, uuid=myuuid), noshow=True)

result_,cumuDf =  testgraphN(g3, cumuDf)
result_.to_pickle("shr/line/line_df_{uuid}.pkl".format(uuid=myuuid))
save_graph(g3, "shr/line/line_graph_{uuid}.pkl".format(uuid=myuuid))
#showdfres(result_)
#showdfres(cumuDf)


In [ ]:
""" TEST for delaunay """
myuuid = uuid.uuid4()

def cifnotexists(pth):
    if not os.path.exists(pth):
        os.makedirs(pth)
cifnotexists('./shr')
cifnotexists('./shr/delaunay')


def delauneygraphres(points=75, clustermin=3, clustermax=5):
    global cumuDf
    cifnotexists('./shr/delaunay/{points}'.format(points=points))
    pth = './shr/delaunay/{points}/'.format(points=points)

    maxsteps = 5
    for ep in range(0, maxsteps-2, 1):
        nodeledges = (1/maxsteps)*ep
        g1 = generateRandomGraph(points, coordinateGenerator=generateRandomCoordinatesShifted, deleteEdges=nodeledges)
        g1ac = AreaCache(g1, defaultdistfn=d_step, pcdistances=True)
        lpp = g1ac.precomputeLongestSP()
        for nocluster in range(clustermin,clustermax):
            cifnotexists(pth+"{nocluster}".format(nocluster=nocluster))
            cifnotexists(pth+"{nocluster}/{nodeledges}".format(nocluster=nocluster, nodeledges=nodeledges))
            result_,cumuDf =  testgraphN(g1, cumuDf, ccluster=nocluster)
            for l in range(1, lpp+1):
                C,E = ExtensionalWeaklyConvexHull2(g1, g1.vs.select(class_eq=0), l, epsilon=0.0, distfn=d_step, debug=False, areacache=g1ac)
                print(len(C), len(E))
                showgraph(g1, classmark=0, recursive=False, saveAs='{pth}{cl}/{nodeledges}/delaunay_{l}_{uuid}.png'.format(pth=pth,cl=nocluster, nodeledges=nodeledges, l=l, uuid=myuuid), noshow=True)
                plotcomponents(g1, C, classmark=0, saveAs="{pth}{cl}/{nodeledges}/delaunay_comp_{l}_{uuid}.png".format(pth=pth,cl=nocluster, nodeledges=nodeledges,l=l, uuid=myuuid), noshow=True)

            result_.to_pickle('{pth}{cl}/{nodeledges}/delaunay_df_{uuid}.pkl'.format(pth=pth, cl=nocluster, nodeledges=nodeledges, uuid=myuuid))
            save_graph(g1, '{pth}{cl}/{nodeledges}/delaunay_graph_{uuid}.pkl'.format(pth=pth, cl=nocluster, nodeledges=nodeledges, uuid=myuuid))

In [ ]:
delauneygraphres(points=75, clustermin=1, clustermax=2)

In [ ]:
delauneygraphres(points=150, clustermin=1, clustermax=2)

In [ ]:
delauneygraphres(points=300, clustermin=1, clustermax=2)

In [ ]:
delauneygraphres(points=75, clustermin=3, clustermax=4)

In [ ]:
delauneygraphres(points=150, clustermin=3, clustermax=4)

In [ ]:
delauneygraphres(points=300, clustermin=3, clustermax=4)